In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.S
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
 <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

# Vertex AI: Create, train, and deploy an AutoML text classification model

## Overview

This notebook walks you through the major phases of building and using an AutoML text classification model on [Vertex AI](https://cloud.google.com/vertex-ai/docs/). 

### Dataset

In this notebook, you use the "Happy Moments" sample dataset to train a model. The resulting model classifies happy moments into categores that reflect the causes of happiness. 

### Objective

In this tutorial, you learn how to use `AutoML` to train a text classification model.

This tutorial uses the following Google Cloud ML services:

- `AutoML Training`
- `Vertex AI Model` resource

The steps performed include:

* Create a `Vertex AI Dataset`.
* Train an `AutoML` text classification `Model` resource.
* Obtain the evaluation metrics for the `Model` resource.
* Create an `Endpoint` resource.
* Deploy the `Model` resource to the `Endpoint` resource.
* Make an online prediction
* Make a batch prediction

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI Training and Serving
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage

## Before you begin

**Note:** This notebook does not require a GPU runtime.

### Set up your local development environment

**If you are using Colab or Workbench AI Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install the following packages for executing this notebook.

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform google-cloud-storage jsonlines -q

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI, BigQuery, Compute Engine and Cloud Storage APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,bigquery,compute_component,storage_component).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: vertex-ai-dev


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [4]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [104]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

1. **Click Create service account**.

2. In the **Service account name** field, enter a name, and click **Create**.

3. In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex AI" into the filter box, and select **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

4. Click Create. A JSON file that contains your key downloads to your local environment.

5. Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [105]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [106]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [107]:
! gsutil mb -l $REGION $BUCKET_URI

E0728 09:01:56.406179345     176 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Creating gs://vertex-ai-devaip-20220728090135/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [108]:
! gsutil ls -al $BUCKET_URI

E0728 09:02:35.929826112     176 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


### Import libraries and define constants

In [10]:
import jsonlines
from google.cloud import aiplatform, storage

### Initialize Vertex AI 

Initialize the Vertex AI SDK for Python for your project.

In [11]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## Create a `Dataset` resource and import your data

The notebook uses the 'Happy Moments' dataset for demonstration purposes. You can change it to another text classification dataset that [conforms to the data preparation requirements](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text#classification).

Using the Python SDK, you create a dataset and import the dataset in one call to `TextDataset.create()`, as shown in the following cell.

Creating and importing data is a long-running operation. This next step can take a while. The `create()` method waits for the operation to complete, outputting statements as the operation progresses. The statements contain the full name of the dataset that you will use in the following section.

**Note**: You can close the noteboook while you wait for this operation to complete. 

In [12]:
# Use a timestamp to ensure unique resources
src_uris = "gs://cloud-ml-data/NL-classification/happiness.csv"
display_name = f"e2e-text-dataset-{TIMESTAMP}"

In [13]:
ds = aiplatform.TextDataset.create(
    display_name=display_name,
    gcs_source=src_uris,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
    sync=True,
)

Creating TextDataset
Create TextDataset backing LRO: projects/931647533046/locations/us-central1/datasets/5685867097322684416/operations/6056760475637514240
TextDataset created. Resource name: projects/931647533046/locations/us-central1/datasets/5685867097322684416
To use this TextDataset in another session:
ds = aiplatform.TextDataset('projects/931647533046/locations/us-central1/datasets/5685867097322684416')
Importing TextDataset data: projects/931647533046/locations/us-central1/datasets/5685867097322684416
Import TextDataset data backing LRO: projects/931647533046/locations/us-central1/datasets/5685867097322684416/operations/1276189471183732736
TextDataset data imported. Resource name: projects/931647533046/locations/us-central1/datasets/5685867097322684416


## Train your text classification model

Once your dataset has finished importing data, you are ready to train your model. To do this, you first need the full resource name of your dataset, where the full name has the format `projects/[YOUR_PROJECT]/locations/[YOUR_REGIO)N]/datasets/[YOUR_DATASET_ID]`. If you don't have the resource name handy, you can list all of the datasets in your project using `TextDataset.list()`. 

As shown in the following code block, you can pass in the display name of your dataset in the call to `list()` to filter the results.


In [14]:
datasets = aiplatform.TextDataset.list(filter=f'display_name="{display_name}"')
print(datasets)

resource name: projects/931647533046/locations/us-central1/datasets/5685867097322684416]


When you create a new model, you need a reference to the `TextDataset` object that corresponds to your dataset. You can use the `ds` variable you created previously when you created the dataset or you can also list all of your datasets to get a reference to your dataset. Each item returned from `TextDataset.list()` is an instance of `TextDataset`.

The following code block shows how to instantiate a `TextDataset` object using a dataset ID. Note that this code is intentionally verbose for demonstration purposes.

In [15]:
# Get the dataset ID if it's not available
dataset_id = "[your-dataset-id]"

if dataset_id == "[your-dataset-id]":
    # Use the reference to the new dataset captured when we created it
    dataset_id = ds.resource_name.split("/")[-1]
    print(f"Dataset ID: {dataset_id}")

text_dataset = aiplatform.TextDataset(dataset_id)

Dataset ID: 5685867097322684416


Now you can begin training your model. Training the model is a two part process:

1. **Define the training job.** You must provide a display name and the type of training you want when you define the training job.
2. **Run the training job.** When you run the training job, you need to supply a reference to the dataset to use for training. At this step, you can also configure the data split percentages.

You do not need to specify [data splits](https://cloud.google.com/vertex-ai/docs/general/ml-use). The training job has a default setting of  training 80%/ testing 10%/ validate 10% if you don't provide these values.

To train your model, you call `AutoMLTextTrainingJob.run()` as shown in the following snippets. The method returns a reference to your new `Model` object.

As with importing data into the dataset, training your model can take a substantial amount of time. The client library prints out operation status messages while the training pipeline operation processes. You must wait for the training process to complete before you can get the resource name and ID of your new model, which is required for model evaluation and model deployment.

**Note**: You can close the notebook while you wait for the operation to complete.

In [27]:
# Define the training job
training_job_display_name = f"e2e-text-training-job-{TIMESTAMP}"
job = aiplatform.AutoMLTextTrainingJob(
    display_name=training_job_display_name,
    prediction_type="classification",
    multi_label=False,
)

In [28]:
model_display_name = f"e2e-text-classification-model-{TIMESTAMP}"

# Run the training job
model = job.run(
    dataset=text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.7,
    validation_fraction_split=0.2,
    test_fraction_split=0.1,
    sync=True,
)

View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8245297345198030848?project=931647533046
AutoMLTextTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/8245297345198030848 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTextTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/8245297345198030848 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTextTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/8245297345198030848 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTextTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/8245297345198030848 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTextTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/8245297345198030848 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTextTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/8245

## Review model evaluation scores

After your model training has finished, you can review the evaluation scores for it using the `list_model_evaluations()` method. This method will return an iterator for each evaluation slice.

In [29]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

{'name': 'projects/931647533046/locations/us-central1/models/2451026922591748096@1/evaluations/8286069572818173952', 'metricsSchemaUri': 'gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml', 'metrics': {'auPrc': 0.95271105, 'confusionMatrix': {'rows': [[177.0, 2.0, 31.0, 0.0, 8.0, 1.0, 32.0], [4.0, 399.0, 17.0, 0.0, 25.0, 1.0, 6.0], [20.0, 7.0, 1108.0, 8.0, 34.0, 8.0, 42.0], [1.0, 1.0, 9.0, 47.0, 0.0, 2.0, 0.0], [3.0, 8.0, 40.0, 0.0, 1322.0, 2.0, 16.0], [3.0, 0.0, 5.0, 0.0, 3.0, 80.0, 7.0], [22.0, 3.0, 76.0, 2.0, 17.0, 6.0, 314.0]], 'annotationSpecs': [{'id': '349744203435081728', 'displayName': 'leisure'}, {'id': '1790896084193640448', 'displayName': 'bonding'}, {'displayName': 'achievement', 'id': '2943817588800487424'}, {'displayName': 'exercise', 'id': '4096739093407334400'}, {'displayName': 'affection', 'id': '6402582102621028352'}, {'id': '7555503607227875328', 'displayName': 'nature'}, {'displayName': 'enjoy_the_moment', 'id': '8708425111834722

## Deploy your text classification model

Once your model has completed training, you must deploy it to an _endpoint_ to get online predictions from it. When you deploy the model to an endpoint, a copy of the model is made on the endpoint with a new resource name and display name.

You can deploy multiple models to the same endpoint and split traffic between the various models assigned to the endpoint. However, you must deploy one model at a time to the endpoint. To change the traffic split percentages, you must assign new values on your second (and subsequent) models each time you deploy a new model.

The following code block demonstrates how to deploy a model. The code snippet relies on the Python SDK to create a new endpoint for deployment. The call to `modely.deploy()` returns a reference to an `Endpoint` object--you need this reference for online predictions in the next section.

In [30]:
deployed_model_display_name = f"e2e-deployed-text-classification-model-{TIMESTAMP}"

endpoint = model.deploy(
    deployed_model_display_name=deployed_model_display_name, sync=True
)

Creating Endpoint
Create Endpoint backing LRO: projects/931647533046/locations/us-central1/endpoints/3673310018725216256/operations/7336345719764156416
Endpoint created. Resource name: projects/931647533046/locations/us-central1/endpoints/3673310018725216256
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/931647533046/locations/us-central1/endpoints/3673310018725216256')
Deploying model to Endpoint : projects/931647533046/locations/us-central1/endpoints/3673310018725216256
Deploy Endpoint model backing LRO: projects/931647533046/locations/us-central1/endpoints/3673310018725216256/operations/765593863430602752
Endpoint model deployed. Resource name: projects/931647533046/locations/us-central1/endpoints/3673310018725216256


In case you didn't record the name of the new endpoint, you can get a list of all your endpoints as you did before with datasets and models. For each endpoint, you can list the models deployed to that endpoint. To get a reference to the model that you just deployed, you can check the `display_name` of each model deployed to the endpoint against the model you're looking for.

In [31]:
endpoints = aiplatform.Endpoint.list()

endpoint_with_deployed_model = []

for endpoint_ in endpoints:
    for model in endpoint_.list_models():
        if model.display_name.find(deployed_model_display_name) == 0:
            endpoint_with_deployed_model.append(endpoint_)

print(endpoint_with_deployed_model)

resource name: projects/931647533046/locations/us-central1/endpoints/3673310018725216256]


## Get online predictions from your model

Now that you have your endpoint's resource name, you can get online predictions from the text classification model. To get the online prediction, you send a prediction request to your endpoint.

In [32]:
endpoint_name = "[your-endpoint-name]"
if endpoint_name == "[your-endpoint-name]":
    endpoint_name = endpoint.resource_name

print(f"Endpoint name: {endpoint_name}")

endpoint = aiplatform.Endpoint(endpoint_name)
content = "I got a high score on my math final!"

response = endpoint.predict(instances=[{"content": content}])

for prediction_ in response.predictions:
    ids = prediction_["ids"]
    display_names = prediction_["displayNames"]
    confidence_scores = prediction_["confidences"]
    for count, id in enumerate(ids):
        print(f"Prediction ID: {id}")
        print(f"Prediction display name: {display_names[count]}")
        print(f"Prediction confidence score: {confidence_scores[count]}")

Endpoint name: projects/931647533046/locations/us-central1/endpoints/3673310018725216256
Prediction ID: 6402582102621028352
Prediction display name: affection
Prediction confidence score: 4.414122304297052e-05
Prediction ID: 2943817588800487424
Prediction display name: achievement
Prediction confidence score: 0.9998748302459717
Prediction ID: 8708425111834722304
Prediction display name: enjoy_the_moment
Prediction confidence score: 3.9748832932673395e-05
Prediction ID: 1790896084193640448
Prediction display name: bonding
Prediction confidence score: 2.3478048660763307e-06
Prediction ID: 349744203435081728
Prediction display name: leisure
Prediction confidence score: 3.584425212466158e-05
Prediction ID: 7555503607227875328
Prediction display name: nature
Prediction confidence score: 2.5153858018711617e-07
Prediction ID: 4096739093407334400
Prediction display name: exercise
Prediction confidence score: 2.8233748707862105e-06


## Get batch predictions from your model

You can get batch predictions from a text classification model without deploying it. You must first format all of your prediction instances (prediction input) in JSONL format and you must store the JSONL file in a Google Cloud Storage bucket. You must also provide a Google Cloud Storage bucket to hold your prediction output.

To start, you must first create your predictions input file in JSONL format. Each line in the JSONL document needs to be formatted like so:

```
{ "content": "gs://sourcebucket/datasets/texts/source_text.txt", "mimeType": "text/plain"}
```

The `content` field in the JSON structure must be a Google Cloud Storage URI to another document that contains the text input for prediction.
[See the documentation for more information.](https://cloud.google.com/ai-platform-unified/docs/predictions/batch-predictions#text)

In [33]:
instances = [
    "We hiked through the woods and up the hill to the ice caves",
    "My kitten is so cute",
]
input_file_name = "batch-prediction-input.jsonl"

For batch prediction, you must supply the following:

+ All of your prediction instances as individual files on Google Cloud Storage, as TXT files for your instances
+ A JSONL file that lists the URIs of all your prediction instances
+ A Google Cloud Storage bucket to hold the output from batch prediction

For this tutorial, the following cells create a new Storage bucket, upload individual prediction instances as text files to the bucket, and then create the JSONL file with the URIs of your prediction instances.

In [114]:
# Instantiate the Storage client and create the new bucket
# from google.cloud import  storage
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
# Iterate over the prediction instances, creating a new TXT file
# for each.
input_file_data = []
for count, instance in enumerate(instances):
    instance_name = f"input_{count}.txt"
    instance_file_uri = f"{BUCKET_URI}/{instance_name}"
    # Add the data to store in the JSONL input file.
    tmp_data = {"content": instance_file_uri, "mimeType": "text/plain"}
    input_file_data.append(tmp_data)

    # Create the new instance file
    blob = bucket.blob(instance_name)
    blob.upload_from_string(instance)

input_str = "\n".join([str(d) for d in input_file_data])
file_blob = bucket.blob(f"{input_file_name}")
file_blob.upload_from_string(input_str)

Now that you have the bucket with the prediction instances ready, you can send a batch prediction rhttps://storage.googleapis.com/upload/storage/v1/b/gs://vertex-ai-devaip-20220728004429/o?uploadType=multipartequest to Vertex AI. When you send a request to the service, you must provide the URI of your JSONL file and your output bucket, including the `gs://` protocols.

With the Python SDK, you can create a batch prediction job by calling `Model.batch_predict()`.

In [146]:
job_display_name = "e2e-text-classification-batch-prediction-job"
model = aiplatform.Model(model_name=model.model)
batch_prediction_job = model.batch_predict(
    job_display_name=job_display_name,
    gcs_source=f"{BUCKET_URI}/{input_file_name}",
    gcs_destination_prefix=f"{BUCKET_URI}/output",
    sync=True,
)
batch_prediction_job_name = batch_prediction_job.resource_name

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/931647533046/locations/us-central1/batchPredictionJobs/369768234322231296
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/931647533046/locations/us-central1/batchPredictionJobs/369768234322231296')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/369768234322231296?project=931647533046
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/369768234322231296 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/369768234322231296 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/369768234322231296 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/931647533046/locations/us-central1/batchPredictionJobs/369768

Once the batch prediction job completes, the Python SDK prints out the resource name of the batch prediction job in the format `projects/[PROJECT_ID]/locations/[LOCATION]/batchPredictionJobs/[BATCH_PREDICTION_JOB_ID]`. You can query the Vertex AI service for the status of the batch prediction job using its ID.

The following code snippet demonstrates how to create an instance of the `BatchPredictionJob` class to review its status. Note that you need the full resource name printed out from the Python SDK for this snippet.


In [147]:
from google.cloud.aiplatform import jobs

batch_job = jobs.BatchPredictionJob(batch_prediction_job_name)
print(f"Batch prediction job state: {str(batch_job.state)}")

Batch prediction job state: JobState.JOB_STATE_SUCCEEDED


After the batch job has completed, you can view the results of the job in your output Storage bucket. You might want to first list all of the files in your output bucket to find the URI of the output file.

In [148]:
BUCKET_OUTPUT = f"{BUCKET_URI}/output"

! gsutil ls -a $BUCKET_OUTPUT

E0728 10:00:29.091150251     176 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


gs://vertex-ai-devaip-20220728090135/output/prediction-e2e-text-classification-model-20220726065605-2022-07-28T09:54:35.262891Z/


The output from the batch prediction job should be contained in a folder (or _prefix_) that includes the name of the batch prediction job plus a time stamp for when it was created.

For example, if your batch prediction job name is `my-job` and your bucket name is `my-bucket`, the URI of the folder containing your output might look like the following:

```
gs://my-bucket/output/prediction-my-job-2021-06-04T19:54:25.889262Z/
```

To read the batch prediction results, you must download the file locally and open the file. The next cell copies all of the files in the `BUCKET_OUTPUT_FOLDER` into a local folder.

In [149]:
RESULTS_DIRECTORY = "prediction_results"
RESULTS_DIRECTORY_FULL = f"{RESULTS_DIRECTORY}/output"

# Create missing directories
os.makedirs(RESULTS_DIRECTORY, exist_ok=True)

# Get the Cloud Storage paths for each result
! gsutil -m cp -r $BUCKET_OUTPUT $RESULTS_DIRECTORY

# Get most recently modified directory
latest_directory = max(
    (
        os.path.join(RESULTS_DIRECTORY_FULL, d)
        for d in os.listdir(RESULTS_DIRECTORY_FULL)
    ),
    key=os.path.getmtime,
)

print(f"Local results folder: {latest_directory}")

E0728 10:01:09.207265811     176 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Copying gs://vertex-ai-devaip-20220728090135/output/prediction-e2e-text-classification-model-20220726065605-2022-07-28T09:54:35.262891Z/predictions_00001.jsonl...
/ [1/1 files][  959.0 B/  959.0 B] 100% Done                                    
Operation completed over 1 objects/959.0 B.                                      
Local results folder: prediction_results/output/prediction-e2e-text-classification-model-20220726065605-2022-07-28T09:54:35.262891Z


With all of the results files downloaded locally, you can open them and read the results. In this tutorial, you use the [`jsonlines`](https://jsonlines.readthedocs.io/en/latest/) library to read the output results.

The following cell opens up the JSONL output file and then prints the predictions for each instance.

In [150]:
# Get downloaded results in directory
results_files = []
for dirpath, _, files in os.walk(latest_directory):
    for file in files:
        if file.find("predictions") >= 0:
            results_files.append(os.path.join(dirpath, file))


# Consolidate all the results into a list
results = []
for results_file in results_files:
    # Open each result
    with jsonlines.open(results_file) as reader:
        for result in reader.iter(type=dict, skip_invalid=True):
            instance = result["instance"]
            prediction = result["prediction"]
            print(f"\ninstance: {instance['content']}")
            for key, output in prediction.items():
                print(f"\n{key}: {output}")


instance: gs://vertex-ai-devaip-20220728090135/input_1.txt

ids: ['8022892806055919616', '1105363778414837760', '6869971301449072640', '3411206787628531712', '4564128292235378688', '5717049796842225664', '2258285283021684736']

displayNames: ['affection', 'achievement', 'enjoy_the_moment', 'bonding', 'leisure', 'nature', 'exercise']

confidences: [0.98783094, 0.009668145, 0.0023065973, 0.0001260451, 3.435437e-05, 1.7344602e-05, 1.6450787e-05]

instance: gs://vertex-ai-devaip-20220728090135/input_0.txt

ids: ['5717049796842225664', '6869971301449072640', '1105363778414837760', '4564128292235378688', '8022892806055919616', '2258285283021684736', '3411206787628531712']

displayNames: ['nature', 'enjoy_the_moment', 'achievement', 'leisure', 'affection', 'exercise', 'bonding']

confidences: [0.47380814, 0.30582574, 0.16660185, 0.050528545, 0.0021286055, 0.00087396626, 0.0002331497]


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

* Dataset
* Training job
* Model
* Endpoint
* Batch prediction
* Batch prediction bucket

In [152]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

batch_job.delete()
endpoint.undeploy_all()
# `force` parameter ensures that models are undeployed before deletion
endpoint.delete(force=True)

model.delete()

text_dataset.delete()

# Training job
job.delete()

Deleting BatchPredictionJob : projects/931647533046/locations/us-central1/batchPredictionJobs/369768234322231296


NotFound: 404 The BatchPredictionJob "projects/931647533046/locations/us-central1/batchPredictionJobs/369768234322231296" does not exist.

## Next Steps

After completing this tutorial, see the following documentation pages to learn more about Vertex AI:

* [Preparing text training data](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text)
* [Training an AutoML model using the API](https://cloud.google.com/vertex-ai/docs/training/automl-api#text)
* [Evaluating AutoML models](https://cloud.google.com/vertex-ai/docs/training/evaluating-automl-models#text)
* [Deploying a model using ther Vertex AI API](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#aiplatform_create_endpoint_sample-python)
* [Getting online predictions from AutoML models](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#aiplatform_create_endpoint_sample-python)
* [Getting batch predictions](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions#text)